# Test the data sets
In this notebook, we perform checks on the generated data set to make sure we stored the correct states, displacements and selected jumps. Similar tests are also present in the unit tests for the LatGas module, but this notebook is also intended to give a more hands-on view into what data we have stored in the dataset.

In [1]:
import numpy as np
import h5py
from tqdm import tqdm
import pickle
from onsager import crystal, supercell, cluster

In [2]:
# Load the data set to check

c0 = 80 # atom percent of slow species
with h5py.File("singleStep_FCC_SR2_c0_{}.h5".format(c0), "r") as fl:
    state1List = np.array(fl["InitStates"])
    state2List = np.array(fl["FinStates"])
    dispList = np.array(fl["SpecDisps"])
    rateList = np.array(fl["rates"])
    jmpSelects = np.array(fl["JumpSelects"])
    rnChecks = np.array(fl["randNums"])
    AllRates_st1 = np.array(fl["AllJumpRates_Init"])
    AllRates_st2 = np.array(fl["AllJumpRates_Fin"])

Ntraj = state1List.shape[0]
print(Ntraj)

20000


In [3]:
specs, Spcounts = np.unique(state1List[0], return_counts=True)
print(specs, Spcounts)

[0 1 2] [408 103   1]


In [4]:
# get the necessary crystal data
with h5py.File("../../CrysDat_FCC/CrystData.h5", "r") as fl:
    lattice = np.array(fl["Lattice_basis_vectors"])
    superlatt = np.array(fl["SuperLatt"])
    dxList = np.array(fl["dxList_1nn"])
    NNsites = np.array(fl["NNsiteList_sitewise"])
    RtoSiteInd = np.array(fl["RToSiteInd"])
    siteIndtoR = np.array(fl["SiteIndToR"])
    JumpNewSites = np.array(fl["JumpSiteIndexPermutation"])

jList = NNsites[1:, 0]
print(jList)

# generate a supercell with the Onsager module
crys = crystal.Crystal(lattice=lattice, basis=[[np.array([0., 0., 0.])]], chemistry=["A"])
print(crys)
superFCC = supercell.ClusterSupercell(crys, superlatt)
Nsites = len(superFCC.mobilepos)
print(Nsites)

[ 15  57 449  71 448  64   7   1   8  56 456 120]
#Lattice:
  a1 = [0.  0.5 0.5]
  a2 = [0.5 0.  0.5]
  a3 = [0.5 0.5 0. ]
#Basis:
  (A) 0.0 = [0. 0. 0.]
512


In [5]:
# begin tests
v = 2 # vacancy species
vacSiteInd = 0
Rvac = np.array([0,0,0])
N_units = 8
SpecRates = np.array([0.001, 1.0])
for traj in tqdm(range(0, Ntraj, 10), position=0, leave=True, ncols=65):
    state1 = state1List[traj]
    state2 = state2List[traj]
    
    specs_1, specNums_1 = np.unique(state1, return_counts=True)
    specs_2, specNums_2 = np.unique(state2, return_counts=True)
    
    # check species conservation
    assert np.array_equal(specNums_1, specNums_2)
    assert np.array_equal(specNums_1, Spcounts)
    
    # check that the vacancy is zero in both states
    assert state1[0] == state2[0] == v
    
    # Next check that the correct displacement was stored for the state selected
    dx = dxList[jmpSelects[traj]]
    assert np.allclose(dispList[traj, v], dx)
    
    # Next, check that displacement was recorded correctly
    RJump, ci = crys.cart2pos(dxList[jmpSelects[traj]])
    assert ci == (0, 0)
    siteJump = superFCC.index(RJump, ci)[0]
    specB = state1[siteJump]
    assert np.allclose(dispList[traj, specB], -dx)
    
    # Next check that state2 has been translated correctly
    assert np.array_equal(state2, state1[JumpNewSites[jmpSelects[traj]]])
    state2UT = state1.copy()
    state2UT[0] = state1[siteJump]
    state2UT[siteJump] = v
    
    for R0 in range(N_units):
        for R1 in range(N_units):
            for R2 in range(N_units):
                R0T = R0 + RJump[0]
                R1T = R1 + RJump[1]
                R2T = R2 + RJump[2]
                
                posR1 = np.array([R0, R1, R2])
                posR1T = np.array([R0T, R1T, R2T])
                site1 = superFCC.index(posR1, (0, 0))[0]
                site2 = superFCC.index(posR1T, (0, 0))[0]
                
                assert state2[site1] == state2UT[site2], "\n{} {} {}".format(R0, R1, R2)
    
    # Next, check that the correct escape rate was computed
    rate = 0.
    for jmp in range(dxList.shape[0]):
        dx = dxList[jmp]
        dxtoR, ci = crys.cart2pos(dx)
        assert ci == (0, 0)
        Jsite = superFCC.index(dxtoR, ci)[0]
        
        spec = state1[Jsite]
        assert np.math.isclose(AllRates_st1[traj, jmp], SpecRates[spec])
        rate += SpecRates[spec]
        
        spec = state2[Jsite]
        assert np.math.isclose(AllRates_st2[traj, jmp], SpecRates[spec])      
        
    assert np.allclose(rate, rateList[traj])
    
    rateProbs = AllRates_st1[traj] / rate
    rateProbs_cumul = np.cumsum(rateProbs)
    
    place = np.searchsorted(rateProbs_cumul, rnChecks[traj])
    assert place == jmpSelects[traj]

100%|████████████████████████| 2000/2000 [00:34<00:00, 58.02it/s]


In [6]:
# check the uncorrelated value - should be close to (1-c0/100)
1-c0/100., np.dot(np.linalg.norm(dispList[:, 1, :], axis=1)**2, rateList)/(6.0 * 20000)

(0.19999999999999996, 0.20085252500000003)